In [3]:
import numpy as np
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (int): Total number of frames to sample.
        frame_sample_rate (int): Sample every n-th frame.
        seg_len (int): Maximum allowed index of sample's last frame.
    Returns:
        indices (List[int]): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [6]:
import numpy as np

# Example parameters
clip_len = 5         # Sample 5 frames
frame_sample_rate = 3  # Every 2nd frame is picked
seg_len = 50        # Total video frames

indices = sample_frame_indices(clip_len, frame_sample_rate, seg_len)
print(indices)

[ 3  6 10 14 17]


In [4]:
import av
import numpy as np

from transformers import VivitImageProcessor, VivitModel
from huggingface_hub import hf_hub_download
import torch

np.random.seed(0)

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [50]:
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

In [54]:
container

<av.InputContainer '/Users/yuexujiang/.cache/huggingface/hub/datasets--nielsr--video-demo/snapshots/657fbe79a427948714ba0f45f454a494d346f75e/eating_spaghetti.mp4'>

In [55]:
indices = sample_frame_indices(clip_len=35, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container=container, indices=indices)

In [56]:
video.shape

(35, 360, 640, 3)

In [57]:
image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
model = VivitModel.from_pretrained("google/vivit-b-16x2-kinetics400")

Some weights of VivitModel were not initialized from the model checkpoint at google/vivit-b-16x2-kinetics400 and are newly initialized: ['vivit.pooler.dense.bias', 'vivit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
# model.config
image_processor

VivitImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "VivitImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "offset": true,
  "resample": 2,
  "rescale_factor": 0.00784313725490196,
  "size": {
    "shortest_edge": 224
  }
}

In [58]:
inputs = image_processor(list(video), return_tensors="pt")

In [59]:
print(inputs['pixel_values'].shape)

inputs_video = np.array(inputs['pixel_values'].squeeze(0).permute(0, 2, 3, 1))
# inputs_video = np.reshape(inputs['pixel_values'], (32, 224, 224, -1))
inputs_video.shape

torch.Size([1, 35, 3, 224, 224])


/var/folders/6n/xf2sm24x5b35mjnd4fq41zm00000gp/T/ipykernel_99007/1521614888.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  inputs_video = np.array(inputs['pixel_values'].squeeze(0).permute(0, 2, 3, 1))


(35, 224, 224, 3)

In [47]:
import av
import cv2  # For visualization

# ... (your existing code for reading the video) ...

# Display the original video using OpenCV
for frame in video:  # 'video' is your numpy array of frames
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # Convert RGB to BGR for OpenCV
    cv2.imshow('Original Video', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'): # Adjust delay (25ms) for frame rate
        break

cv2.destroyAllWindows()
container.close() # Close the video file

# Display the original video using OpenCV
for frame in inputs_video:  # 'video' is your numpy array of frames
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # Convert RGB to BGR for OpenCV
    cv2.imshow('Original Video', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'): # Adjust delay (25ms) for frame rate
        break

cv2.destroyAllWindows()
container.close() # Close the video file



In [60]:
inputs_video

array([[[[-2.4980392 , -2.764706  , -2.9686275 ],
         [-2.4196079 , -2.7490196 , -2.937255  ],
         [-2.372549  , -2.7333333 , -2.9058824 ],
         ...,
         [-1.6823529 , -1.1647059 , -2.0431373 ],
         [-1.6980392 , -1.1803921 , -2.0588236 ],
         [-1.7137254 , -1.1960784 , -2.0745099 ]],

        [[-2.5137255 , -2.764706  , -2.9686275 ],
         [-2.4196079 , -2.764706  , -2.937255  ],
         [-2.372549  , -2.7333333 , -2.9058824 ],
         ...,
         [-1.6980392 , -1.1490196 , -2.027451  ],
         [-1.7137254 , -1.1647059 , -2.0431373 ],
         [-1.7294117 , -1.1803921 , -2.0588236 ]],

        [[-2.4980392 , -2.764706  , -2.9686275 ],
         [-2.4509804 , -2.7490196 , -2.937255  ],
         [-2.3882353 , -2.717647  , -2.890196  ],
         ...,
         [-1.6980392 , -1.1490196 , -2.027451  ],
         [-1.7137254 , -1.1647059 , -2.0431373 ],
         [-1.7294117 , -1.1803921 , -2.0588236 ]],

        ...,

        [[ 0.7019608 , -0.9607842 , -1